In [1]:
import numpy as np
import pandas as pd
import tweepy as tw
import json

_keys_file = open('tokens.json')
keys = json.load(_keys_file)

### Authenticate

In [3]:
auth = tw.OAuthHandler(keys['api_key'], keys['api_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

### Get Tweets

In [4]:
search_words = "#covid19 -filter:retweets"
date_since = "2022-01-01"

# Collect tweets
tweetsResults = tw.Cursor(
  api.search_tweets,
  q=search_words,
  lang="es",
).items(1500)

In [5]:
tweets = []
for tweet in tweetsResults:
  tweets.append(tweet)
print("Total Tweets fetched:", len(tweets))

Total Tweets fetched: 1500


In [6]:
# Init dataframe
data = []
columns = ['userName', 'userLocation', 'userDescription', 'isUserVerified', 'date', 'text', 'source']

for tweet in tweets:
  try:text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
  except: pass

  # Adding new data
  data.append([
    tweet.user.name, 
    tweet.user.location,
    tweet.user.description,
    tweet.user.verified,
    tweet.created_at,
    text, 
    tweet.source
  ])

# show the dataframe
df = pd.DataFrame(columns=columns, data=data)

Rate limit reached. Sleeping for: 673


In [7]:
df.head()

,userName,userLocation,userDescription,isUserVerified,date,text,source
0,OaxacaesNoticia,,Diario de información general. Reporteros de l...,False,2022-09-10 02:14:04+00:00,"Informa la @SSalud_mx que existen 7,056,164 ca...",Twitter for iPhone
1,@ojopolitico,,Periodista en activo desde 1978,False,2022-09-10 02:13:59+00:00,"Informa la @SSalud_mx que existen 7,056,164 ca...",Twitter for iPhone
2,Fernando Cruz López,"Oaxaca de Juárez, Oaxaca",Periodista activo desde 1978. Director General...,False,2022-09-10 02:13:52+00:00,"Informa la @SSalud_mx que existen 7,056,164 ca...",Twitter for iPhone
3,Juan Carlos Mori,Perú,"Medico Anestesiólogo, Congresista de la Repúbl...",False,2022-09-10 02:11:51+00:00,"Hoy en la Comisión #COVID19, el viceministro d...",Twitter for Android
4,Secretaría de Salud Jalisco,"Guadalajara, Jalisco","Nos encargamos de la salud en Jalisco, coordin...",False,2022-09-10 02:09:14+00:00,#CorteDiario \nEste es el panorama de #COVID19...,Twitter for Android


In [9]:
df.to_csv('train.csv', index=False, encoding='utf-8')